#### Introduction to Statistical Learning, Lab 4.3

# Linear Discriminant Analysis


We will now perform a linear discrimant ana;usis (LDA) non the `Smarket` data set, trying to predict `Direction` using `Lag1` and `Lag2`. We go beyond the now familiar `statsmodels` library and use the `sklearn` library.



In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn import metrics
from islpy import datasets, utils, lmplots
sns.set()
%matplotlib inline

We first load the data set.

In [4]:
smarket = datasets.Smarket()
smarket.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
